### Supported devices
一般的系统都由多个计算devices。 TensorFlow支持CPU和GPU。 他们用字符串代表，例如：
* "/cpu:0": The CPU of your machine.
* "/device:GPU:0": The GPU of your machine, if you have one.
* "/device:GPU:1": The second GPU of your machine, etc.

如果说TensorFlow的operation在CPU和GPU上面均可以工作，那么GPU会拥有优先权。

### Logging Device placement
找到operations和tensors被分配到哪个devices，创建一个拥有 __log_device_placement__ 配置选择为 __True__ 的session。
示例如下：

In [2]:
import tensorflow as tf
# Create a graph:
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config = tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))

# the output will see in the terminal:
Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla K40c, pci bus
id: 0000:05:00.0
b: /job:localhost/replica:0/task:0/device:GPU:0
a: /job:localhost/replica:0/task:0/device:GPU:0
MatMul: /job:localhost/replica:0/task:0/device:GPU:0
[[ 22.  28.]
 [ 49.  64.]]

[[ 22.  28.]
 [ 49.  64.]]


### Manual device placement
如果不想让系统自动分配operation到device，可以通过 __with tf.device__ 创建一个device 文本，这样所有的在这个文本当中的operations都会被分配到同一个device.  
用例如下：

从这个例子的输出当中，你可以看到 a,b都被分配到 cpu:0, 由于device没有显示的指定 MatMul operation分配到哪个device，所以TensorFlow运行中会将这个operation选择一个可分配的device进行分配，并且在需要的时候自动复制这些tensors

In [5]:
# create a graph
with tf.device("/cpu:0"):
    a=tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b=tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a,b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config = tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))

# the output will see in the terminal
Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla K40c, pci bus
id: 0000:05:00.0
b: /job:localhost/replica:0/task:0/cpu:0
a: /job:localhost/replica:0/task:0/cpu:0
MatMul: /job:localhost/replica:0/task:0/device:GPU:0
[[ 22.  28.]
 [ 49.  64.]]

[[ 22.  28.]
 [ 49.  64.]]


'Device mapping:\n/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla K40c, pci bus\nid: 0000:05:00.0\nb: /job:localhost/replica:0/task:0/cpu:0\na: /job:localhost/replica:0/task:0/cpu:0\nMatMul: /job:localhost/replica:0/task:0/device:GPU:0\n[[ 22.  28.]\n [ 49.  64.]]'

### Allowing GPU memory growth
默认情况下， TensorFlow把所有GPU(受制于(subject to) CUDA_VISIBLE_DEVICES)几乎全部的内存映射到process中。通过相对的使用GPU珍贵的内存可以更加有效，并且能够减少内存碎片化。

在一些情况下，只需要为process分配可提供内存的一部分内容，或者是仅仅在process需要的时候增加内存的使用。TensorFlow在Session中提供两种 Config options来控制这个选择。

第一个是 __allow_growth__ 选项，这个试图在依照runtime allcations来分配足够多的CPU的内存: 它首先分配非常少的内存，然后当Session运行的时候，分配所需要的更多的内存，通过TensorFlow process所需要的GPU 内存区间来增长。 注意我们并不释放内存，因为那样可能会导致更加严重的内存碎片。执行这个选择，通过 __tf.ConfigProto().gpu_options.allow_growth=True__ 来执行，详情见示例1：

第二个方法是 __per_process_gpu_memory_fraction__ 选择。 这决定了可见的GPU所有内存被分配的比例。例如，你可以告诉TensorFlow分配GPU的所有内存的40%通过示例2完成：
这种方法在你知道TensorFlow process需要多少的内存数量的时候是十分有用的。

In [ ]:
# 1:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config,...)

# 2:
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4
session = tf.Session(config=config,...)

### Using a single GPU on a multi-GPU system
在系统上有多个GPU，默认情况下，拥有最低ID值的GPU会被选择。如果说你想要在不同的GPU上面运行，那么，你需要显示的指定你的需求，如下面示例1：

如果你指定的GPU不存在，那么你会得到一个 __InvalidArgumentError__: ，如下面示例2：

如果想要 TensorFlow 自动选择一个存在的并且支持的device来运行这些operations以防止特定的device不存在，可以在创建session的时候在配置选项中设定 __allow_soft_placement=True__ ，如下面示例3：

In [ ]:
# 1
# Creates a graph.
with tf.device('/device:GPU:2'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))

# 不存在device时，错误显示
InvalidArgumentError: Invalid argument: Cannot assign a device to node 'b':
Could not satisfy explicit device specification '/device:GPU:2'
   [[Node: b = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [3,2]
   values: 1 2 3...>, _device="/device:GPU:2"]()]]
    
# 3：
# Creates a graph.
with tf.device('/device:GPU:2'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)
# Creates a session with allow_soft_placement and log_device_placement set
# to True.
sess = tf.Session(config=tf.ConfigProto(
      allow_soft_placement=True, log_device_placement=True))
# Runs the op.
print(sess.run(c))

### Using multiple GPUs
当在多GPUs环境下运行TensorFlow时，可以通过 multi-tower来构建模型，这里每个tower都分配给不同的GPU，例如：

In [ ]:
# Creates a graph.
c = []
for d in ['/device:GPU:2', '/device:GPU:3']:
    with tf.device(d):
        a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3])
        b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2])
        c.append(tf.matmul(a, b))
with tf.device('/cpu:0'):
    sum = tf.add_n(c)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(sum))

## output of this code
Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla K20m, pci bus
id: 0000:02:00.0
/job:localhost/replica:0/task:0/device:GPU:1 -> device: 1, name: Tesla K20m, pci bus
id: 0000:03:00.0
/job:localhost/replica:0/task:0/device:GPU:2 -> device: 2, name: Tesla K20m, pci bus
id: 0000:83:00.0
/job:localhost/replica:0/task:0/device:GPU:3 -> device: 3, name: Tesla K20m, pci bus
id: 0000:84:00.0
Const_3: /job:localhost/replica:0/task:0/device:GPU:3
Const_2: /job:localhost/replica:0/task:0/device:GPU:3
MatMul_1: /job:localhost/replica:0/task:0/device:GPU:3
Const_1: /job:localhost/replica:0/task:0/device:GPU:2
Const: /job:localhost/replica:0/task:0/device:GPU:2
MatMul: /job:localhost/replica:0/task:0/device:GPU:2
AddN: /job:localhost/replica:0/task:0/cpu:0
[[  44.   56.]
 [  98.  128.]]